# Imports

In [ ]:
!pip install -q evaluate rouge_score

import numpy as np
from evaluate import load
from tqdm import tqdm

!pip install --upgrade evaluate rouge_score

from datasets import load_dataset
import evaluate
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
!pip install evaluate rouge_score
from evaluate import load
import numpy as np

In [ ]:
rouge = load("rouge")

# Dataset

In [ ]:
!pip install --upgrade datasets

from datasets import load_dataset
dataset = load_dataset("midas/inspec", "raw")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

inspec.py:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/538k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/283k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/272k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
train_sample = dataset["train"][0]
print("Fields in the sample: ", [key for key in train_sample.keys()])
print("Tokenized Document: ", train_sample["document"])
print("Document BIO Tags: ", train_sample["doc_bio_tags"])
print("Extractive/present Keyphrases: ", train_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", train_sample["abstractive_keyphrases"])
print("\n-----------\n")

# sample from the validation split
print("Sample from validation dataset split")
validation_sample = dataset["validation"][0]
print("Fields in the sample: ", [key for key in validation_sample.keys()])
print("Tokenized Document: ", validation_sample["document"])
print("Document BIO Tags: ", validation_sample["doc_bio_tags"])
print("Extractive/present Keyphrases: ", validation_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", validation_sample["abstractive_keyphrases"])
print("\n-----------\n")

# sample from the test split
print("Sample from test dataset split")
test_sample = dataset["test"][0]
print("Fields in the sample: ", [key for key in test_sample.keys()])
print("Tokenized Document: ", test_sample["document"])
print("Document BIO Tags: ", test_sample["doc_bio_tags"])
print("Extractive/present Keyphrases: ", test_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", test_sample["abstractive_keyphrases"])
print("\n-----------\n")

Fields in the sample:  ['id', 'document', 'doc_bio_tags', 'extractive_keyphrases', 'abstractive_keyphrases', 'other_metadata']
Tokenized Document:  ['A', 'conflict', 'between', 'language', 'and', 'atomistic', 'information', 'Fred', 'Dretske', 'and', 'Jerry', 'Fodor', 'are', 'responsible', 'for', 'popularizing', 'three', 'well-known', 'theses', 'in', 'contemporary', 'philosophy', 'of', 'mind', ':', 'the', 'thesis', 'of', 'Information-Based', 'Semantics', '-LRB-', 'IBS', '-RRB-', ',', 'the', 'thesis', 'of', 'Content', 'Atomism', '-LRB-', 'Atomism', '-RRB-', 'and', 'the', 'thesis', 'of', 'the', 'Language', 'of', 'Thought', '-LRB-', 'LOT', '-RRB-', '.', 'LOT', 'concerns', 'the', 'semantically', 'relevant', 'structure', 'of', 'representations', 'involved', 'in', 'cognitive', 'states', 'such', 'as', 'beliefs', 'and', 'desires', '.', 'It', 'maintains', 'that', 'all', 'such', 'representations', 'must', 'have', 'syntactic', 'structures', 'mirroring', 'the', 'structure', 'of', 'their', 'contents

# Model

## As is model

In [ ]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_docs = [" ".join(doc) for doc in dataset["test"]["document"]]

In [ ]:
def generate_outputs(text_list, prefix="", max_len=64):
    outputs = []
    batch_size = 2
    for i in tqdm(range(0, len(text_list), batch_size)):
        batch = text_list[i:i + batch_size]
        batch_inputs = [prefix + text for text in batch]
        tokenized = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512)
        tokenized = {k: v.to(device) for k, v in tokenized.items()}
        output_tokens = model.generate(**tokenized, max_length=max_len)
        decoded = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
        outputs.extend(decoded)
    return outputs

In [ ]:
print("Generating summaries...")
generated_summaries = generate_outputs(test_docs, max_len=64)

print("Generating abstractive keyphrases...")
generated_keyphrases = generate_outputs(test_docs, prefix="keyphrases: ", max_len=32)

# Show sample output
sample_idx = 0
print("\n-----------")
print("Sample Document:")
print(test_docs[sample_idx])

print("\nGenerated Summary:")
print(generated_summaries[sample_idx])

print("\nGenerated Abstractive Keyphrases:")
print(generated_keyphrases[sample_idx])

print("\nGround Truth Abstractive Keyphrases:")
print("; ".join(dataset["test"][sample_idx]["abstractive_keyphrases"]))

Generating summaries...


100%|██████████| 250/250 [04:01<00:00,  1.04it/s]


Generating abstractive keyphrases...


100%|██████████| 250/250 [03:16<00:00,  1.27it/s]


-----------
Sample Document:
A new graphical user interface for fast construction of computation phantoms and MCNP calculations : application to calibration of in vivo measurement systems Reports on a new utility for development of computational phantoms for Monte Carlo calculations and data analysis for in vivo measurements of radionuclides deposited in tissues . The individual properties of each worker can be acquired for a rather precise geometric representation of his -LRB- her -RRB- anatomy , which is particularly important for low energy gamma ray emitting sources such as thorium , uranium , plutonium and other actinides . The software enables automatic creation of an MCNP input data file based on scanning data . The utility includes segmentation of images obtained with either computed tomography or magnetic resonance imaging by distinguishing tissues according to their signal -LRB- brightness -RRB- and specification of the source and detector . In addition , a coupling of indiv

In [ ]:
reference_keyphrases = ["; ".join(k) for k in dataset["test"]["abstractive_keyphrases"]]
rouge = load_metric("rouge")
results = rouge.compute(predictions=generated_keyphrases, references=reference_keyphrases)
print(results)

NameError: name 'load_metric' is not defined

## Fine tuned model